### 1- Import test data


In [856]:
from dotenv import load_dotenv
import os
import pprint

import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import pytz
import numpy as np
import time

from pybit.unified_trading import HTTP

load_dotenv()


#connect to bYBIT

session = HTTP(
    testnet=False,
    demo=True,
    api_key=os.getenv('API_DEMO_KEY'),
    api_secret=os.getenv('API_DEMO_SECRET'),
)

### Bybit methods

In [857]:
from datetime import datetime
import time
def get_account_balance(ticker):
    # Retrieve account balance from Bybit
    response = session.get_wallet_balance(accountType="UNIFIED",coin=ticker)  

    
    if response['retCode'] == 0:
        balance = response['result']['list'][0]['coin'][0]['walletBalance']
        print(f'The balance of {ticker} is: {balance}')
        return balance
    else:
        print("Failed to retrieve account balance:", response['ret_msg'])
        return None
    



def fetch_market_data(symbol, interval, category, start_date=None, end_date=None ):
    #setup time parameters
    if start_date:
        start_date = int(pd.to_datetime(start_date).timestamp()*1000)
 

    if end_date:
        end_date = int(pd.to_datetime(end_date).timestamp()*1000)
    #print(start_ts, end_ts, start_date, end_date)
    #Setup graph
    data = session.get_kline(symbol=symbol, interval=interval, category = category,limit = 1000, start= start_date
    )
    df = pd.DataFrame(data['result']['list'])
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']

    df['Date'] = pd.to_datetime(pd.to_numeric(df['Date']), unit='ms')
    df.set_index('Date', inplace=True)
    melbourne_tz = pytz.timezone('Australia/Melbourne')
    df.index = df.index.tz_localize('UTC').tz_convert(melbourne_tz)
    df.index = df.index.tz_localize(None)

    df = df[::-1]

    #add relevant indicators
    df['RSI'] = ta.rsi(df['Close'].astype(float), length = 14)
    df['Fast_EMA'] = ta.ema(df['Close'].astype(float), length=30)
    df['Slow_EMA'] = ta.ema(df['Close'].astype(float), length=50)

    df['ATR'] = ta.atr(df['High'].astype(float), df['Low'].astype(float),df['Close'].astype(float), length=7)

    bbands = ta.bbands(df['Close'].astype(float), length = 20, std = 2)
    df = df.join(bbands)


    backcandles = 6
    #df.reset_index(inplace=True, drop=True)
    df['EMA_SIGNAL'] = [ema_signal(df, i) if i >= backcandles - 1 else 0 for i in range(len(df))]
    df['TOTAL_SIGNAL'] = [total_signal(df, i) if i >= backcandles-1 else 0 for i in range(len(df))]


    df['pointpos'] = [
    float(row['Low']) - (float(row['High']) - float(row['Low'])) * 0.5 if row['TOTAL_SIGNAL'] == 1 else  # LONG
    float(row['High']) + (float(row['High']) - float(row['Low'])) * 0.5 if row['TOTAL_SIGNAL'] == -1 else  # SHORT
    None
    for _, row in df.iterrows()

]
    return df

       
    

def ema_signal(df, current_candle, backcandles=6 ):
    df_copy = df.copy()
    start = max(0, current_candle - backcandles+1) #starts at 0, or whatever candle we can reach . ADD 1 if we want to include current candle right?
    end = current_candle+1      #add 1 ehre too
    df_new = df_copy.iloc[start:end]

    if all(df_new['Fast_EMA'] > df_new['Slow_EMA']):
        return 1  # Uptrend
    elif all(df_new['Fast_EMA'] < df_new['Slow_EMA']):
        return -1  # Downtrend
    else:
        return 0  
     

def total_signal(df, current_candle, backcandles = 6):

    #if EMA signal is uptrend and we close under bollinger band lower, we return a BUY signal
    if (ema_signal(df, current_candle, backcandles)==1 and df['Close'].astype(float).iloc[current_candle]<=df['BBL_20_2.0'].iloc[current_candle]
    ):
        return 1
    
    
    if (ema_signal(df, current_candle, backcandles)==-1 and df['Close'].astype(float).iloc[current_candle]>=df['BBU_20_2.0'].iloc[current_candle]
    ):
        return -1
    return 0




### Retrieve market data

In [858]:
symbol = 'SOLUSDT'
interval = '5'
category = 'linear'
start_date = '2024-10-01 00:00:00'
end_date = '2023-02-01 00:00:00'


df = fetch_market_data(symbol,interval, category, start_date, None)

 
df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].apply(pd.to_numeric)

print(len(df))
display(df[df['TOTAL_SIGNAL']!= 0].tail(30))

#display(df.tail(30))
#print(df.dtypes)


fig = go.Figure(data = go.Candlestick(x=df.index,
                                      open = df['Open'],
                                      high = df['High'],
                                      low = df['Low'],
                                      close = df['Close']))
#fig.update_layout(xaxis_rangeslider_visible=False)

fig.add_trace(go.Scatter(x=df.index, y=df['Fast_EMA'], line=dict(color='blue'), name='Fast EMA (9)'))
fig.add_trace(go.Scatter(x=df.index, y=df['Slow_EMA'], line=dict(color='red'), name='Slow EMA (21)'))

fig.add_trace(go.Scatter(x=df.index, y=df['BBU_20_2.0'], line=dict(color='green', width = 1), name='Upper Band'))
fig.add_trace(go.Scatter(x=df.index, y=df['BBL_20_2.0'], line=dict(color='orange', width = 1), name='Lower Band'))


fig.add_scatter(x= df.index, y=df['pointpos'], mode='markers', marker=dict(size=5, color = "MediumPurple"), name = "entry")
fig.update_layout(
    width=1200,  
    height=800  
)

fig.show()


1000


,Open,High,Low,Close,Volume,Turnover,RSI,Fast_EMA,Slow_EMA,ATR,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,EMA_SIGNAL,TOTAL_SIGNAL,pointpos
Date,,,,,,,,,,,,,,,,,,
2024-10-01 21:55:00,156.83,156.83,156.44,156.47,33277.2,5211118.735,40.513024,156.784815,156.652958,0.252791,156.514767,156.8900,157.265233,0.478339,-0.059652,1,1,156.245
2024-10-01 22:40:00,156.21,156.30,156.02,156.05,36832.1,5750766.771,31.070410,156.590920,156.566220,0.285257,156.110606,156.5895,157.068394,0.611655,-0.063278,1,1,155.880
2024-10-02 15:25:00,146.53,146.65,146.13,146.34,20419.0,2986614.531,43.244915,146.718884,146.533499,0.408421,146.380450,146.8020,147.223550,0.574310,-0.047978,1,1,145.870
2024-10-02 18:05:00,147.70,147.74,147.08,147.17,42593.7,6279657.032,41.345766,147.650141,147.373605,0.443580,147.334874,148.0510,148.767126,0.967404,-0.115115,1,1,146.750
2024-10-02 19:40:00,146.73,146.80,146.21,146.42,65294.1,9559901.903,29.494202,147.458127,147.411202,0.415203,146.526108,147.5120,148.497892,1.336695,-0.053813,1,1,145.915
2024-10-02 19:45:00,146.42,146.50,145.51,145.84,132847.3,19377480.058,23.941705,147.353732,147.349587,0.497317,146.224837,147.4455,148.666163,1.655749,-0.157634,1,1,145.015
2024-10-03 02:55:00,146.38,146.44,145.91,145.97,25394.2,3710160.533,41.248531,146.586497,146.377424,0.530380,146.117303,147.0715,148.025697,1.297595,-0.077187,1,1,145.645
2024-10-03 03:00:00,145.97,146.03,145.37,145.37,57424.5,8365110.782,35.563755,146.508014,146.337917,0.548897,145.775392,146.9820,148.188608,1.641845,-0.167988,1,1,145.040
2024-10-03 03:30:00,145.72,145.74,144.75,144.76,79997.2,11618088.315,33.189140,146.232456,146.199328,0.565015,144.888927,146.5105,148.132073,2.213593,-0.039754,1,1,144.255


### Backtesting

In [859]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
def SIGNAL():
    return df['TOTAL_SIGNAL']


class Bollinger_EMA(Strategy):
    mysize = 0.1
    slcoef = 1.1 # Reduce stop-loss coefficient
    TPSLRatio = 1.5 # Reduce take-profit ratio
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        
        slatr = self.slcoef * self.data.ATR[-1]
        
        TPSLRatio = self.TPSLRatio
  
        if self.signal1[-1]==1 and len(self.trades)==0 :
 
           #long position
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr * TPSLRatio
            #print(f"Long SL={sl1}, TP={tp1}, Entry={self.data.Close[-1]} at {self.data.index[-1]}")
            self.buy(sl=sl1, tp=tp1, size = self.mysize )

            
        elif self.signal1[-1]==-1 and len(self.trades)==0:       
            #Short position
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr * TPSLRatio
            #print(f"Short SL={sl1}, TP={tp1}, Entry={self.data.Close[-1]}")
            self.sell(sl=sl1, tp=tp1, size = self.mysize)
        
bt = Backtest(df, Bollinger_EMA, cash=1000, margin=1/50)
bt.run()


c:\Users\Eunha\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:127: RuntimeWarning:

divide by zero encountered in double_scalars



Start                     2024-10-01 10:00:00
End                       2024-10-04 21:15:00
Duration                      3 days 11:15:00
Exposure Time [%]                        13.9
Equity Final [$]                   1335.98062
Equity Peak [$]                    1335.98062
Return [%]                           33.59806
Buy & Hold Return [%]                -8.27487
Return (Ann.) [%]           27043810572.96803
Volatility (Ann.) [%]       27293530052.78856
CAGR [%]                    137788054193.9783
Sharpe Ratio                          0.99085
Sortino Ratio                             inf
Calmar Ratio                 3879289767.56713
Max. Drawdown [%]                    -6.97133
Avg. Drawdown [%]                    -2.03213
Max. Drawdown Duration        0 days 20:10:00
Avg. Drawdown Duration        0 days 03:28:00
# Trades                                   21
Win Rate [%]                         66.66667
Best Trade [%]                        1.05665
Worst Trade [%]                   

In [666]:
bt.plot()

GridPlot(id='p3352', ...)